# Constants:

In [1]:
SEQ_LEN = 240   # how many past candles to use to predict
CANDLES_SHIFT = 5 # how many candles to shift between sequences
NAME = "shift5_seq240_ov40th004"
VALIDATION_PCT = 0.2

In [2]:
import pickle

with open(f"data/pkl/{NAME}/{NAME}_train_x.pkl", "rb") as file:
    train_x = pickle.load(file)
with open(f"data/pkl/{NAME}/{NAME}_train_y.pkl", "rb") as file:
    train_y = pickle.load(file)
with open(f"data/pkl/{NAME}/{NAME}_validation_x.pkl", "rb") as file:
    validation_x = pickle.load(file)
with open(f"data/pkl/{NAME}/{NAME}_validation_y.pkl", "rb") as file:
    validation_y = pickle.load(file)

# Model:

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from tqdm.keras import TqdmCallback
import datetime
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import regularizers
import json

c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
EPOCHS = 200
batchSize = 96

layers = 2
nodes_arr = [256] # [128, 256] #128
denseNodes = 128

dropOut_arr = [0.9]#[0.5, 0.75, 0.9] #0.88 0.66
rec_dropout = 0
l1l2_reg_arr = [5e-4, 1e-5]#1e-3 #1e-6

learningRate = 0.00001
decay = 1e-06 #1e-06

for nodes in nodes_arr:
   for dropOut in dropOut_arr:
      for l1l2_reg in l1l2_reg_arr:



        hp_suffix = f"bs({batchSize})_layers({layers})_noded({nodes})_dNodes({denseNodes})_do({int(dropOut*100)}%)_recdo({rec_dropout})_l1l2({l1l2_reg})_lr({learningRate})_decay({decay})"



        # model
        model = Sequential()

        for _ in range(layers-1):
          model.add(Bidirectional(LSTM(nodes, 
                                       activation="tanh", 
                                       recurrent_activation = 'sigmoid', 
                                       recurrent_dropout = rec_dropout, 
                                       unroll = False, 
                                       use_bias = True, 
                                       input_shape=(train_x.shape[1:]), 
                                       return_sequences=True,
                                       kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                                       #bias_regularizer=regularizers.l2(l1l2_reg),
                                       activity_regularizer=regularizers.l2(l1l2_reg)
                                       )))
          model.add(Dropout(dropOut))
          #model.add(BatchNormalization())

        model.add(Bidirectional(LSTM(nodes, 
                                     activation="tanh", 
                                     recurrent_activation = 'sigmoid', 
                                     recurrent_dropout = rec_dropout, 
                                     unroll = False, 
                                     use_bias = True, 
                                     input_shape=(train_x.shape[1:]),
                                     kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                                     #bias_regularizer=regularizers.l2(l1l2_reg),
                                     activity_regularizer=regularizers.l2(l1l2_reg)
                                     )))
        model.add(Dropout(dropOut))
        #model.add(BatchNormalization())

        model.add(Dense(denseNodes, 
                        activation="relu",
                        kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                        #bias_regularizer=regularizers.l2(l1l2_reg), 
                        activity_regularizer=regularizers.l2(l1l2_reg)))
        model.add(Dropout(dropOut))

        model.add(Dense(3, activation="softmax"))



        # opt  
        opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?


        model.compile(loss="sparse_categorical_crossentropy",
                      optimizer = opt,
                      metrics=["accuracy"])


        filename = "bi_" + NAME + "-{epoch:02d}"
        filepath = f"models/{filename}.h5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch

        # Prepare TensorBoard callback
        log_dir = f"logs/fit/bi_{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


        # store hyper-parameters
        # Path to the JSON file
        file_path = "models/hps/hps.json"
        # Load the JSON file into a dictionary
        with open(file_path, "r") as json_file:
            data = json.load(json_file)
        # Add a new key-value pair to the dictionary
        data[f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")] = hp_suffix
        # Write the updated dictionary back to the file
        with open(file_path, "w") as json_file:
            json.dump(data, json_file, indent=4)




        # train
        history = model.fit(
          train_x, train_y,
          batch_size = batchSize,
          epochs = EPOCHS,
          validation_data=(validation_x, validation_y),
          callbacks=[checkpoint, TqdmCallback(verbose=0), tensorboard_callback])
          #callbacks=[checkpoint])

c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
  0%|          | 0/200 [00:00<?, ?epoch/s]

Epoch 1/200
412/412 [==============================] - ETA: 0s - loss: 4.0008 - accuracy: 0.3451

  0%|          | 1/200 [01:29<4:56:51, 89.50s/epoch, loss=4, accuracy=0.345, val_loss=3.69, val_accuracy=0.499]

412/412 [==============================] - 90s 204ms/step - loss: 4.0008 - accuracy: 0.3451 - val_loss: 3.6859 - val_accuracy: 0.4988
Epoch 2/200
412/412 [==============================] - ETA: 0s - loss: 3.8321 - accuracy: 0.3529

  1%|          | 2/200 [02:51<4:41:26, 85.28s/epoch, loss=3.83, accuracy=0.353, val_loss=3.58, val_accuracy=0.504]

...

100%|█████████▉| 199/200 [4:30:49<01:21, 81.60s/epoch, loss=0.941, accuracy=0.603, val_loss=0.925, val_accuracy=0.607]

412/412 [==============================] - 82s 198ms/step - loss: 0.9406 - accuracy: 0.6034 - val_loss: 0.9253 - val_accuracy: 0.6074
Epoch 200/200
412/412 [==============================] - ETA: 0s - loss: 0.9439 - accuracy: 0.6028

100%|██████████| 200/200 [4:32:11<00:00, 81.59s/epoch, loss=0.944, accuracy=0.603, val_loss=0.922, val_accuracy=0.614]

412/412 [==============================] - 82s 198ms/step - loss: 0.9439 - accuracy: 0.6028 - val_loss: 0.9220 - val_accuracy: 0.6140


100%|██████████| 200/200 [4:32:11<00:00, 81.66s/epoch, loss=0.944, accuracy=0.603, val_loss=0.922, val_accuracy=0.614]
